In [24]:
import torch
from torch import nn
from torch import Tensor
import requests
import os
from pathlib import Path
from typing import Callable
from functools import partial


URL = 'https://download.pytorch.org/models/resnet18-5c106cde.pth'

In [68]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [70]:
from glasses.nn.models.classification.efficientnet import SEInvertedResidualBlock, EfficientNetEncoder


SEInvertedResidualBlock(32, 64)

SEInvertedResidualBlock(
  (block): Residual(
    (block): Sequential(
      (exp): ConvBnAct(
        (conv): Conv2dPad(32, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU6()
      )
      (conv): Sequential(
        (0): ConvBnAct(
          (conv): DepthWiseConv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=192, bias=False)
          (bn): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): ReLU6()
        )
        (1): Sequential(
          (0): SEModuleConv(
            (avg_pool): AdaptiveAvgPool2d(output_size=1)
            (att): Sequential(
              (0): Conv2d(192, 1, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (1): ReLU(inplace=True)
              (2): Conv2d(1, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (3): Sigmoid()
            )
          )

In [71]:
EfficientNetEncoder()

EfficientNetEncoder(
  (gate): Sequential(
    (0): ConvBnAct(
      (conv): Conv2dPad(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): Swish()
    )
  )
  (blocks): ModuleList(
    (0): EfficientNetLayer(
      (block): Sequential(
        (0): SEInvertedResidualBlock(
          (block): Residual(
            (block): Sequential(
              (exp): ConvBnAct(
                (conv): Conv2dPad(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (bn): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (act): ReLU6()
              )
              (conv): Sequential(
                (0): ConvBnAct(
                  (conv): DepthWiseConv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=96, bias=False)
                  (bn): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, tra

In [57]:
res = Residual(nn.Sequential(nn.Conv2d(32, 64, 3, padding=1)), shortcut=nn.Conv2d(32, 64, 3, padding=1), 
               res_func = lambda a, b: a.add_(b))

res(torch.rand(1,32, 48, 48))

RuntimeError: The size of tensor a (32) must match the size of tensor b (64) at non-singleton dimension 1

In [62]:
from torchvision.models import mobilenet_v2
mobilenet_v2(False)

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [61]:
from efficientnet_pytorch import EfficientNet
from torchsummary import summary


model = EfficientNet.from_pretrained('efficientnet-b0')

summary(model.cuda(), (3,224,224))

Loaded pretrained weights for efficientnet-b0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 226, 226]               0
Conv2dStaticSamePadding-2         [-1, 32, 112, 112]             864
       BatchNorm2d-3         [-1, 32, 112, 112]              64
MemoryEfficientSwish-4         [-1, 32, 112, 112]               0
         ZeroPad2d-5         [-1, 32, 114, 114]               0
Conv2dStaticSamePadding-6         [-1, 32, 112, 112]             288
       BatchNorm2d-7         [-1, 32, 112, 112]              64
MemoryEfficientSwish-8         [-1, 32, 112, 112]               0
          Identity-9             [-1, 32, 1, 1]               0
Conv2dStaticSamePadding-10              [-1, 8, 1, 1]             264
MemoryEfficientSwish-11              [-1, 8, 1, 1]               0
         Identity-12              [-1, 8, 1, 1]               0
Conv2dStaticSamePadding-13        

In [10]:
model

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=

In [7]:
from glasses.nn.models.classification import MobileNetV2

model  = MobileNetV2()


summary(model.cuda(), (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Conv2dPad-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             ReLU6-3         [-1, 32, 112, 112]               0
   DepthWiseConv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             ReLU6-6         [-1, 32, 112, 112]               0
         Conv2dPad-7         [-1, 16, 112, 112]             512
       BatchNorm2d-8         [-1, 16, 112, 112]              32
InvertedResidualBlock-9         [-1, 16, 112, 112]               0
   MobileNetLayer-10         [-1, 16, 112, 112]               0
        Conv2dPad-11         [-1, 96, 112, 112]           1,536
      BatchNorm2d-12         [-1, 96, 112, 112]             192
            ReLU6-13         [-1, 96, 112, 112]               0
  DepthWiseConv2d-14           [-1, 

In [3]:
from __future__ import annotations
import torch
import torch.nn.functional as F
from torch import nn
from torch import Tensor
from collections import OrderedDict
from typing import List
from functools import partial
from glasses.nn.blocks import ConvBnAct, Conv2dPad
from glasses.nn.models.segmentation.unet import UNet


In [88]:
from glasses.nn.models.classification import ResNet
from glasses.nn.models.segmentation.unet import  UNet

resnet34 = ResNet.resnet18()
# we need to change the first conv in order to accept a gray image
resnet34.encoder.blocks[0].block[0].block.block.conv1 = Conv2dPad(1, 64, kernel_size=1)

unet = UNet(1, n_classes=2, blocks_sizes=resnet34.encoder.blocks_sizes)

unet.encoder.blocks = resnet34.encoder.blocks

x = torch.rand((1, 1, 32 * 12, 32*12))
unet(x).shape

# unet.decoder

torch.Size([1, 2, 384, 384])

In [77]:
from glasses.nn.models.classification.senet import SENetBasicBlock

block = SENetBasicBlock

unet = UNet(down_block=SENetBasicBlock, up_block=SENetBasicBlock)

x = torch.rand((1, 1, 32 * 12, 32*12))
unet(x).shape

torch.Size([1, 2, 384, 384])

In [87]:
unet = UNet(n_classes=4)
unet(x).shape

torch.Size([1, 4, 384, 384])

In [66]:
[r.shape for r in unet.residuals]

[torch.Size([1, 256, 96, 96]),
 torch.Size([1, 128, 192, 192]),
 torch.Size([1, 64, 384, 384])]

In [34]:
from glasses.nn.models.classification.mobilenet import InvertedResidualBlock
from glasses.nn.models.classification.se import SEModule


In [11]:
UpBlock(32, 64)

UpBlock(
  (up): ConvTranspose2d(32, 64, kernel_size=(2, 2), stride=(2, 2))
  (block): Sequential(
    (0): Sequential(
      (0): ConvBnAct(
        (conv): Conv2dPad(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU()
      )
      (1): ConvBnAct(
        (conv): Conv2dPad(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU()
      )
    )
  )
)